In [276]:
# Import Modules
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
import pyspark
import time
import lyricsgenius
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import billboard
import time
import requests
import tweepy
import re
import GetOldTweets3 as got
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.tokenize import word_tokenize
from string import punctuation

from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.ml.feature import CountVectorizer , IDF
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import RandomForest
from pyspark.mllib.evaluation import BinaryClassificationMetrics

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
from gensim import corpora, models
from pprint import pprint
from collections import Counter

APP_NAME = "Random Forest"
SPARK_URL = "local[*]"

spark = SparkSession.builder \
    .appName(APP_NAME) \
    .master(SPARK_URL) \
    .getOrCreate()

[nltk_data] Downloading package stopwords to /Users/pk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/pk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
%env SPOTIPY_CLIENT_ID=28aa32e2bfbc4a2183b24afc6e52a40c
%env SPOTIPY_CLIENT_SECRET=48c3512efc3f4098bdf80304e0a67bab

env: SPOTIPY_CLIENT_ID=28aa32e2bfbc4a2183b24afc6e52a40c
env: SPOTIPY_CLIENT_SECRET=48c3512efc3f4098bdf80304e0a67bab


In [12]:
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

In [134]:
try:
    trackids = []
    for val in sp.playlist_tracks('37i9dQZF1DWUa8ZRTfalHk', fields='items').values():
        for x in val:
            for key, res in x.items():
                if(key == 'track'):
                    for x, vals in res.items():
                        if x == 'id':
                            trackids.append(vals)
    for val in sp.playlist_tracks('37i9dQZF1DX4JAvHpjipBk', fields='items').values():
        for x in val:
            for key, res in x.items():
                if(key == 'track'):
                    for x, vals in res.items():
                        if x == 'id':
                            if vals not in trackids:
                                trackids.append(vals)
except:
    print('error')
print(len(trackids))
print(len(tracks))

error
166
83


In [193]:
chart = billboard.ChartData('hot-100')
tracks = []

for track in trackids:
    Week = dt.datetime.now().strftime('%m/%d/%Y')
    Song = None
    Peak = None
    Performer = []
    SongID = None
    WeeksOnChart = None
    spotify_track_explicit = None
    spotify_track_duration_ms = None
    spotify_track_popularity = None
    danceability = None
    energy = None
    key = None
    loudness = None
    mode = None
    speechiness = None
    acousticness = None
    instrumentalness = None
    liveness = None
    valence = None
    tempo = None
    time_signature = None
    debut = None
    for k, v in sp.track(track).items():
        if k == 'artists':
            for artist in v:
                for key, val in artist.items():
                    if key == 'name':
                        Performer.append(val)
        if k == 'duration_ms':
            spotify_track_duration_ms = v
        if k == 'explicit':
            spotify_track_explicit = v
        if k == 'popularity':
            spotify_track_popularity = v
        if k == 'name':
            Song = re.split("(\(feat.*\))", v)[0].strip()
    if sp.audio_features(track)[0]:
        for k, v in sp.audio_features(track)[0].items():
            if k == 'danceability':
                danceability = v
            if k == 'energy':
                energy = v
            if k == 'key':
                key = v
            if k == 'loudness':
                loudness = v
            if k == 'mode':
                mode = v
            if k == 'speechiness':
                speechiness = v
            if k == 'acousticness':
                acousticness = v
            if k == 'instrumentalness':
                instrumentalness = v
            if k == 'liveness':
                liveness = v
            if k == 'valence':
                valence = v
            if k == 'tempo':
                tempo = v
            if k == 'time_signature':
                time_signature = v
    for song in chart:
        if (Song == song.title and Performer[0] in song.artist):
            Performer[0] = song.artist
            if song.peakPos:
                if song.rank == song.peakPos:
                    Peak = song.rank
                else:
                    Peak = 'N/A'
            else:
                Peak = song.rank
            if song.lastPos:
                if (song.lastPos != song.peakPos and song.lastPos > song.rank):
                    debut = song.lastPos
                else:
                    debut = 'N/A'
            else:
                debut = song.rank
            WeeksOnChart = song.weeks
            
    if debut != 'N/A' and Peak != 'N/A':
        SongID = Song+Performer[0]
        tracks.append([Week, SongID, Song, Performer[0], Peak, WeeksOnChart, spotify_track_explicit, spotify_track_duration_ms, spotify_track_popularity, \
                      danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, time_signature, debut])
    
tracks    

retrying after...1secs
retrying after...1secs
retrying after...2secs


[['05/15/20',
  'DaisiesKaty Perry',
  'Daisies',
  'Katy Perry',
  None,
  None,
  False,
  173123,
  76,
  0.639,
  0.546,
  1,
  -5.382,
  1,
  0.0407,
  0.0837,
  0,
  0.122,
  0.149,
  122.179,
  4,
  None],
 ['05/15/20',
  'Five More MinutesJonas Brothers',
  'Five More Minutes',
  'Jonas Brothers',
  None,
  None,
  False,
  150366,
  65,
  0.836,
  0.304,
  4,
  -7.948,
  1,
  0.0666,
  0.305,
  0,
  0.0979,
  0.688,
  94.037,
  4,
  None],
 ['05/15/20',
  'Lose SomebodyKygo',
  'Lose Somebody',
  'Kygo',
  None,
  None,
  False,
  199549,
  75,
  0.58,
  0.586,
  1,
  -6.883,
  1,
  0.0357,
  0.344,
  0,
  0.0755,
  0.507,
  147.988,
  4,
  None],
 ['05/15/20',
  'Play DateMelanie Martinez',
  'Play Date',
  'Melanie Martinez',
  None,
  None,
  True,
  179867,
  88,
  0.68,
  0.729,
  5,
  -5.077,
  1,
  0.0475,
  0.612,
  0,
  0.224,
  0.446,
  123.97,
  4,
  None],
 ['05/15/20',
  'good guysLANY',
  'good guys',
  'LANY',
  None,
  None,
  False,
  224994,
  71,
  0.455,
  

In [194]:
newtracks = pd.DataFrame(tracks, columns =['Week', 'SongID', 'Song', 'Performer', 'Peak', 'WeeksOnChart', 'spotify_track_explicit', 'spotify_track_duration_ms', 'spotify_track_popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'debut']) 



In [195]:
newtracks

,Week,SongID,Song,Performer,Peak,WeeksOnChart,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,debut
0,05/15/20,DaisiesKaty Perry,Daisies,Katy Perry,NaN,NaN,False,173123,76,0.639,...,-5.382,1.0,0.0407,0.083700,0.000000,0.1220,0.1490,122.179,4.0,NaN
1,05/15/20,Five More MinutesJonas Brothers,Five More Minutes,Jonas Brothers,NaN,NaN,False,150366,65,0.836,...,-7.948,1.0,0.0666,0.305000,0.000000,0.0979,0.6880,94.037,4.0,NaN
2,05/15/20,Lose SomebodyKygo,Lose Somebody,Kygo,NaN,NaN,False,199549,75,0.580,...,-6.883,1.0,0.0357,0.344000,0.000000,0.0755,0.5070,147.988,4.0,NaN
3,05/15/20,Play DateMelanie Martinez,Play Date,Melanie Martinez,NaN,NaN,True,179867,88,0.680,...,-5.077,1.0,0.0475,0.612000,0.000000,0.2240,0.4460,123.970,4.0,NaN
4,05/15/20,good guysLANY,good guys,LANY,NaN,NaN,False,224994,71,0.455,...,-7.719,1.0,0.0751,0.593000,0.000000,0.0899,0.2480,169.919,4.0,NaN
5,05/15/20,Stuck with U (with Justin Bieber)Ariana Grande,Stuck with U (with Justin Bieber),Ariana Grande,NaN,NaN,False,228482,92,0.597,...,-6.658,1.0,0.0418,0.223000,0.000000,0.3820,0.5370,178.765,3.0,NaN
6,05/15/20,Party GirlStaySolidRocky,Party Girl,StaySolidRocky,64.0,2.0,False,147800,86,0.728,...,-9.966,0.0,0.0622,0.749000,0.000000,0.0996,0.6290,130.022,4.0,84.0
7,05/15/20,Be Kind (with Halsey)Marshmello,Be Kind (with Halsey),Marshmello,NaN,NaN,False,172761,89,0.630,...,-7.088,1.0,0.0457,0.007760,0.000000,0.0541,0.4520,93.995,4.0,NaN
8,05/15/20,End this (L.O.V.E.)Hailee Steinfeld,End this (L.O.V.E.),Hailee Steinfeld,NaN,NaN,False,183218,70,0.621,...,-9.733,0.0,0.3330,0.323000,0.000008,0.1350,0.4080,192.183,4.0,NaN
9,05/15/20,X (with KAROL G)Jonas Brothers,X (with KAROL G),Jonas Brothers,NaN,NaN,False,185277,72,0.715,...,-5.090,0.0,0.0617,0.048500,0.000036,0.3210,0.7500,117.977,4.0,NaN


In [196]:
newtracks["Lyrics"] = 0

In [197]:
genius = lyricsgenius.Genius("lRSmSx2mp4ZU4ZovVyII_bDmIN2ROTR_0yU9HDCZI47VKNkZw-sBf5b4pruTYMam")
genius.remove_section_headers = True
genius.verbose = False
genius.skip_non_songs = True

In [198]:
for idx, values in newtracks.iterrows():
    try:
        lyrics = genius.search_song(values['Song'],values['Performer']).lyrics
        newtracks["Lyrics"][idx] = lyrics
    except:
        print("An exception occurred")

/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


An exception occurred
An exception occurred


In [199]:
newtracks.drop(newtracks[newtracks['Lyrics'] == 0].index, inplace = True)

In [200]:
newtracks["LyricPositive"] = 0
newtracks["LyricNeutral"] = 0
newtracks["LyricNegative"] = 0

In [201]:
sia = SentimentIntensityAnalyzer()
for idx, values in newtracks.iterrows():
    if detect(values['Lyrics']) == 'en':
        num_positive = 0
        num_neutral = 0
        num_negative = 0
        for line in values['Lyrics'].splitlines():
            if line != "":
                comp = sia.polarity_scores(line)   
                comp = comp['compound']
                if comp >= 0.5:
                    num_positive += 1
                elif comp > -0.5 and comp < 0.5:
                    num_neutral += 1
                else:
                    num_negative += 1
        num_total = num_negative + num_neutral + num_positive
        percent_negative = (num_negative/float(num_total))*100
        percent_neutral = (num_neutral/float(num_total))*100
        percent_positive = (num_positive/float(num_total))*100
        newtracks['LyricPositive'][idx] = percent_positive
        newtracks['LyricNeutral'][idx] = percent_neutral
        newtracks['LyricNegative'][idx] = percent_negative
newtracks

/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Week,SongID,Song,Performer,Peak,WeeksOnChart,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,...,instrumentalness,liveness,valence,tempo,time_signature,debut,Lyrics,LyricPositive,LyricNeutral,LyricNegative
0,05/15/20,DaisiesKaty Perry,Daisies,Katy Perry,NaN,NaN,False,173123,76,0.639,...,0.000000,0.1220,0.1490,122.179,4.0,NaN,"Told them your dreams, and they all started la...",2,97,0
1,05/15/20,Five More MinutesJonas Brothers,Five More Minutes,Jonas Brothers,NaN,NaN,False,150366,65,0.836,...,0.000000,0.0979,0.6880,94.037,4.0,NaN,"Give me five more minutes\nBaby, I'm not finis...",2,97,0
2,05/15/20,Lose SomebodyKygo,Lose Somebody,Kygo,NaN,NaN,False,199549,75,0.580,...,0.000000,0.0755,0.5070,147.988,4.0,NaN,"It's a classic ""me"" mistake\nSomeone gives me ...",17,82,0
3,05/15/20,Play DateMelanie Martinez,Play Date,Melanie Martinez,NaN,NaN,True,179867,88,0.680,...,0.000000,0.2240,0.4460,123.970,4.0,NaN,"You call me on the telephone, you feel so far ...",0,83,16
4,05/15/20,good guysLANY,good guys,LANY,NaN,NaN,False,224994,71,0.455,...,0.000000,0.0899,0.2480,169.919,4.0,NaN,I wanna call you up\nTell you how I feel\nBut ...,8,91,0
5,05/15/20,Stuck with U (with Justin Bieber)Ariana Grande,Stuck with U (with Justin Bieber),Ariana Grande,NaN,NaN,False,228482,92,0.597,...,0.000000,0.3820,0.5370,178.765,3.0,NaN,"Mmm\nHey, yeah\n(That's just for fun)\n(What?)...",0,78,21
6,05/15/20,Party GirlStaySolidRocky,Party Girl,StaySolidRocky,64.0,2.0,False,147800,86,0.728,...,0.000000,0.0996,0.6290,130.022,4.0,84.0,"Girls just wanna have fun\nSo they party, so t...",46,53,0
7,05/15/20,Be Kind (with Halsey)Marshmello,Be Kind (with Halsey),Marshmello,NaN,NaN,False,172761,89,0.630,...,0.000000,0.0541,0.4520,93.995,4.0,NaN,Drake - Pain 1993 (with Playboi Carti)\nMarshm...,10,84,5
8,05/15/20,End this (L.O.V.E.)Hailee Steinfeld,End this (L.O.V.E.),Hailee Steinfeld,NaN,NaN,False,183218,70,0.621,...,0.000008,0.1350,0.4080,192.183,4.0,NaN,L is for the way you lied to me\nO is I'm the ...,37,60,2
9,05/15/20,X (with KAROL G)Jonas Brothers,X (with KAROL G),Jonas Brothers,NaN,NaN,False,185277,72,0.715,...,0.000036,0.3210,0.7500,117.977,4.0,NaN,Bazzi - Paradise\nJonas Brothers - Cool\nKhali...,8,90,1


In [202]:
newtracks['LyricSentimentTotal'] = newtracks.apply(lambda row: row.LyricPositive + row.LyricNeutral + row.LyricNegative, axis=1)
newtracks.drop(newtracks[newtracks['LyricSentimentTotal'] == 0].index, inplace = True)
newtracks = newtracks.drop(columns=['LyricSentimentTotal'])
newtracks

,Week,SongID,Song,Performer,Peak,WeeksOnChart,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,...,instrumentalness,liveness,valence,tempo,time_signature,debut,Lyrics,LyricPositive,LyricNeutral,LyricNegative
0,05/15/20,DaisiesKaty Perry,Daisies,Katy Perry,NaN,NaN,False,173123,76,0.639,...,0.000000,0.1220,0.1490,122.179,4.0,NaN,"Told them your dreams, and they all started la...",2,97,0
1,05/15/20,Five More MinutesJonas Brothers,Five More Minutes,Jonas Brothers,NaN,NaN,False,150366,65,0.836,...,0.000000,0.0979,0.6880,94.037,4.0,NaN,"Give me five more minutes\nBaby, I'm not finis...",2,97,0
2,05/15/20,Lose SomebodyKygo,Lose Somebody,Kygo,NaN,NaN,False,199549,75,0.580,...,0.000000,0.0755,0.5070,147.988,4.0,NaN,"It's a classic ""me"" mistake\nSomeone gives me ...",17,82,0
3,05/15/20,Play DateMelanie Martinez,Play Date,Melanie Martinez,NaN,NaN,True,179867,88,0.680,...,0.000000,0.2240,0.4460,123.970,4.0,NaN,"You call me on the telephone, you feel so far ...",0,83,16
4,05/15/20,good guysLANY,good guys,LANY,NaN,NaN,False,224994,71,0.455,...,0.000000,0.0899,0.2480,169.919,4.0,NaN,I wanna call you up\nTell you how I feel\nBut ...,8,91,0
5,05/15/20,Stuck with U (with Justin Bieber)Ariana Grande,Stuck with U (with Justin Bieber),Ariana Grande,NaN,NaN,False,228482,92,0.597,...,0.000000,0.3820,0.5370,178.765,3.0,NaN,"Mmm\nHey, yeah\n(That's just for fun)\n(What?)...",0,78,21
6,05/15/20,Party GirlStaySolidRocky,Party Girl,StaySolidRocky,64.0,2.0,False,147800,86,0.728,...,0.000000,0.0996,0.6290,130.022,4.0,84.0,"Girls just wanna have fun\nSo they party, so t...",46,53,0
7,05/15/20,Be Kind (with Halsey)Marshmello,Be Kind (with Halsey),Marshmello,NaN,NaN,False,172761,89,0.630,...,0.000000,0.0541,0.4520,93.995,4.0,NaN,Drake - Pain 1993 (with Playboi Carti)\nMarshm...,10,84,5
8,05/15/20,End this (L.O.V.E.)Hailee Steinfeld,End this (L.O.V.E.),Hailee Steinfeld,NaN,NaN,False,183218,70,0.621,...,0.000008,0.1350,0.4080,192.183,4.0,NaN,L is for the way you lied to me\nO is I'm the ...,37,60,2
9,05/15/20,X (with KAROL G)Jonas Brothers,X (with KAROL G),Jonas Brothers,NaN,NaN,False,185277,72,0.715,...,0.000036,0.3210,0.7500,117.977,4.0,NaN,Bazzi - Paradise\nJonas Brothers - Cool\nKhali...,8,90,1


In [144]:
# Creates New Twitter Dataset with Sentiment Fields
twitter = newtracks[['Week', 'Song', 'Performer', 'SongID']]
twitter['Tweets'] = 0
twitter['TweetPositive'] = 0
twitter['TweetNeutral'] = 0
twitter['TweetNegative'] = 0
twitter

/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

,Week,Song,Performer,SongID,Tweets,TweetPositive,TweetNeutral,TweetNegative
0,05/15/20,Daisies,Katy Perry,DaisiesKaty Perry,0,0,0,0
1,05/15/20,Five More Minutes,Jonas Brothers,Five More MinutesJonas Brothers,0,0,0,0
2,05/15/20,Lose Somebody,Kygo,Lose SomebodyKygo,0,0,0,0
3,05/15/20,Play Date,Melanie Martinez,Play DateMelanie Martinez,0,0,0,0
4,05/15/20,good guys,LANY,good guysLANY,0,0,0,0
5,05/15/20,Stuck with U (with Justin Bieber),Ariana Grande,Stuck with U (with Justin Bieber)Ariana Grande,0,0,0,0
6,05/15/20,Party Girl,StaySolidRocky,Party GirlStaySolidRocky,0,0,0,0
7,05/15/20,Be Kind (with Halsey),Marshmello,Be Kind (with Halsey)Marshmello,0,0,0,0
8,05/15/20,End this (L.O.V.E.),Hailee Steinfeld,End this (L.O.V.E.)Hailee Steinfeld,0,0,0,0
9,05/15/20,X (with KAROL G),Jonas Brothers,X (with KAROL G)Jonas Brothers,0,0,0,0


In [149]:
# Twitter Data Collection
iteration = 0
for idx, values in twitter.iterrows():
    if idx > 32:
        if iteration == 15:
            time.sleep(500) # Makes the operation sleep for 8 minutes as to not overload API with a large number of requests
            iteration = 0
        tweets = []
        d = dt.timedelta(days=14)
        enddate = dt.datetime.strptime(values['Week'], "%m/%d/%y")
        startdate = enddate-d
        tweetCriteria = got.manager.TweetCriteria().setQuerySearch(values['Song'] + " " + values['Performer']).setSince(startdate.strftime("%Y-%m-%d")).setUntil(enddate.strftime("%Y-%m-%d")).setMaxTweets(100)
        tweet = got.manager.TweetManager.getTweets(tweetCriteria)
        stopwords12 = set(stopwords.words('english') + list(punctuation))
        for x in tweet:
            x = x.text
            x = x.lower() # convert text to lower-case
            x = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', x) # remove URLs
            x = re.sub('@[^\s]+', '', x) # remove usernames
            x = re.sub(r'#([^\s]+)', r'\1', x) # remove the # in #hashtag
            try:
                # Checks if the tweets are in english for sentiment analysis
                if detect(x) == 'en':
                    tweets.append(x)
            except:
                print("LangDetect Error")
        twitter['Tweets'][idx] = tweets
        iteration+=1

/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


LangDetect Error
LangDetect Error
LangDetect Error
LangDetect Error
New Iteration
New Iteration
New Iteration
New Iteration
New Iteration
New Iteration
LangDetect Error
LangDetect Error
LangDetect Error
New Iteration
New Iteration


In [151]:
sia = SentimentIntensityAnalyzer()
for idx, values in twitter.iterrows():
    newtweets = []
    song = values['Song'][0].lower()
    performer = values['Performer'][0].lower()
    for x in values['Tweets']:
        text = x.replace(song, "").replace(performer, "").replace('-','').strip()
        if text:
            newtweets.append(text)
    twitter['Tweets'][idx] = newtweets
    num_positive = 0
    num_neutral = 0
    num_negative = 0
    for x in newtweets:
        if x != "":
            comp = sia.polarity_scores(x)   
            comp = comp['compound']
            if comp >= 0.5:
                num_positive += 1
            elif comp > -0.5 and comp < 0.5:
                num_neutral += 1
            else:
                num_negative += 1
    num_total = num_negative + num_neutral + num_positive
    if num_total != 0:
        percent_negative = (num_negative/float(num_total))*100
        percent_neutral = (num_neutral/float(num_total))*100
        percent_positive = (num_positive/float(num_total))*100
        twitter['TweetPositive'][idx] = percent_positive
        twitter['TweetNeutral'][idx] = percent_neutral
        twitter['TweetNegative'][idx] = percent_negative

/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingW

In [164]:
twitter['TweetSentimentTotal'] = twitter.apply(lambda row: row.TweetPositive + row.TweetNeutral + row.TweetNegative, axis=1)

/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [165]:
twitter

,Week,Song,Performer,SongID,Tweets,TweetPositive,TweetNeutral,TweetNegative,TweetSentimentTotal
0,05/15/20,Daisies,Katy Perry,DaisiesKaty Perry,[oay im trying not to sleep but this is me rig...,57,42,0,99
1,05/15/20,Five More Minutes,Jonas Brothers,Five More MinutesJonas Brothers,[exactly 5 hours until x and ive more minutes!...,21,73,5,99
2,05/15/20,Lose Somebody,Kygo,Lose SomebodyKygo,"[we’ve just added ose somebody by ygo, onerepu...",2,97,0,99
3,05/15/20,Play Date,Melanie Martinez,Play DateMelanie Martinez,[ale usic to songs: 224 (+3) (new eak): lay da...,6,84,9,99
4,05/15/20,good guys,LANY,good guysLANY,"[i just wanna ive u everythin, show u i’m a so...",15,78,6,99
5,05/15/20,Stuck with U (with Justin Bieber),Ariana Grande,Stuck with U (with Justin Bieber)Ariana Grande,"[hey cn you plee ply ""tuck with u"" by nd i'...",12,82,5,99
6,05/15/20,Party Girl,StaySolidRocky,Party GirlStaySolidRocky,"[don’t be cared to arty, don’t be cared to art...",7,89,3,99
7,05/15/20,Be Kind (with Halsey),Marshmello,Be Kind (with Halsey)Marshmello,[also e kind y halsey and arshello. literally ...,83,16,0,99
8,05/15/20,End this (L.O.V.E.),Hailee Steinfeld,End this (L.O.V.E.)Hailee Steinfeld,[l is for t way you lid to m o is i'm t only o...,6,91,2,99
9,05/15/20,X (with KAROL G),Jonas Brothers,X (with KAROL G)Jonas Brothers,[love their fans so much. they have really bee...,24,75,0,99


In [203]:
newtracks = pd.merge(newtracks, twitter, how='inner', on=['SongID'])
newtracks

,Week_x,SongID,Song_x,Performer_x,Peak,WeeksOnChart,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,...,LyricNeutral,LyricNegative,Week_y,Song_y,Performer_y,Tweets,TweetPositive,TweetNeutral,TweetNegative,TweetSentimentTotal
0,05/15/20,DaisiesKaty Perry,Daisies,Katy Perry,NaN,NaN,False,173123,76,0.639,...,97,0,05/15/20,Daisies,Katy Perry,[oay im trying not to sleep but this is me rig...,57,42,0,99
1,05/15/20,Five More MinutesJonas Brothers,Five More Minutes,Jonas Brothers,NaN,NaN,False,150366,65,0.836,...,97,0,05/15/20,Five More Minutes,Jonas Brothers,[exactly 5 hours until x and ive more minutes!...,21,73,5,99
2,05/15/20,Lose SomebodyKygo,Lose Somebody,Kygo,NaN,NaN,False,199549,75,0.580,...,82,0,05/15/20,Lose Somebody,Kygo,"[we’ve just added ose somebody by ygo, onerepu...",2,97,0,99
3,05/15/20,Play DateMelanie Martinez,Play Date,Melanie Martinez,NaN,NaN,True,179867,88,0.680,...,83,16,05/15/20,Play Date,Melanie Martinez,[ale usic to songs: 224 (+3) (new eak): lay da...,6,84,9,99
4,05/15/20,good guysLANY,good guys,LANY,NaN,NaN,False,224994,71,0.455,...,91,0,05/15/20,good guys,LANY,"[i just wanna ive u everythin, show u i’m a so...",15,78,6,99
5,05/15/20,Stuck with U (with Justin Bieber)Ariana Grande,Stuck with U (with Justin Bieber),Ariana Grande,NaN,NaN,False,228482,92,0.597,...,78,21,05/15/20,Stuck with U (with Justin Bieber),Ariana Grande,"[hey cn you plee ply ""tuck with u"" by nd i'...",12,82,5,99
6,05/15/20,Party GirlStaySolidRocky,Party Girl,StaySolidRocky,64.0,2.0,False,147800,86,0.728,...,53,0,05/15/20,Party Girl,StaySolidRocky,"[don’t be cared to arty, don’t be cared to art...",7,89,3,99
7,05/15/20,Be Kind (with Halsey)Marshmello,Be Kind (with Halsey),Marshmello,NaN,NaN,False,172761,89,0.630,...,84,5,05/15/20,Be Kind (with Halsey),Marshmello,[also e kind y halsey and arshello. literally ...,83,16,0,99
8,05/15/20,End this (L.O.V.E.)Hailee Steinfeld,End this (L.O.V.E.),Hailee Steinfeld,NaN,NaN,False,183218,70,0.621,...,60,2,05/15/20,End this (L.O.V.E.),Hailee Steinfeld,[l is for t way you lid to m o is i'm t only o...,6,91,2,99
9,05/15/20,X (with KAROL G)Jonas Brothers,X (with KAROL G),Jonas Brothers,NaN,NaN,False,185277,72,0.715,...,90,1,05/15/20,X (with KAROL G),Jonas Brothers,[love their fans so much. they have really bee...,24,75,0,99


In [204]:
newtracks = newtracks.drop(columns=['Week_y', 'Song_y', 'Performer_y'])
newtracks = newtracks.rename(columns={"Week_x": "Week", "Song_x": "Song", "Performer_x": "Performer"})
newtracks.drop(newtracks[newtracks['TweetSentimentTotal'] == 0].index, inplace = True)
newtracks = newtracks.drop(columns=['TweetSentimentTotal'])
newtracks


,Week,SongID,Song,Performer,Peak,WeeksOnChart,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,...,time_signature,debut,Lyrics,LyricPositive,LyricNeutral,LyricNegative,Tweets,TweetPositive,TweetNeutral,TweetNegative
0,05/15/20,DaisiesKaty Perry,Daisies,Katy Perry,NaN,NaN,False,173123,76,0.639,...,4.0,NaN,"Told them your dreams, and they all started la...",2,97,0,[oay im trying not to sleep but this is me rig...,57,42,0
1,05/15/20,Five More MinutesJonas Brothers,Five More Minutes,Jonas Brothers,NaN,NaN,False,150366,65,0.836,...,4.0,NaN,"Give me five more minutes\nBaby, I'm not finis...",2,97,0,[exactly 5 hours until x and ive more minutes!...,21,73,5
2,05/15/20,Lose SomebodyKygo,Lose Somebody,Kygo,NaN,NaN,False,199549,75,0.580,...,4.0,NaN,"It's a classic ""me"" mistake\nSomeone gives me ...",17,82,0,"[we’ve just added ose somebody by ygo, onerepu...",2,97,0
3,05/15/20,Play DateMelanie Martinez,Play Date,Melanie Martinez,NaN,NaN,True,179867,88,0.680,...,4.0,NaN,"You call me on the telephone, you feel so far ...",0,83,16,[ale usic to songs: 224 (+3) (new eak): lay da...,6,84,9
4,05/15/20,good guysLANY,good guys,LANY,NaN,NaN,False,224994,71,0.455,...,4.0,NaN,I wanna call you up\nTell you how I feel\nBut ...,8,91,0,"[i just wanna ive u everythin, show u i’m a so...",15,78,6
5,05/15/20,Stuck with U (with Justin Bieber)Ariana Grande,Stuck with U (with Justin Bieber),Ariana Grande,NaN,NaN,False,228482,92,0.597,...,3.0,NaN,"Mmm\nHey, yeah\n(That's just for fun)\n(What?)...",0,78,21,"[hey cn you plee ply ""tuck with u"" by nd i'...",12,82,5
6,05/15/20,Party GirlStaySolidRocky,Party Girl,StaySolidRocky,64.0,2.0,False,147800,86,0.728,...,4.0,84.0,"Girls just wanna have fun\nSo they party, so t...",46,53,0,"[don’t be cared to arty, don’t be cared to art...",7,89,3
7,05/15/20,Be Kind (with Halsey)Marshmello,Be Kind (with Halsey),Marshmello,NaN,NaN,False,172761,89,0.630,...,4.0,NaN,Drake - Pain 1993 (with Playboi Carti)\nMarshm...,10,84,5,[also e kind y halsey and arshello. literally ...,83,16,0
8,05/15/20,End this (L.O.V.E.)Hailee Steinfeld,End this (L.O.V.E.),Hailee Steinfeld,NaN,NaN,False,183218,70,0.621,...,4.0,NaN,L is for the way you lied to me\nO is I'm the ...,37,60,2,[l is for t way you lid to m o is i'm t only o...,6,91,2
9,05/15/20,X (with KAROL G)Jonas Brothers,X (with KAROL G),Jonas Brothers,NaN,NaN,False,185277,72,0.715,...,4.0,NaN,Bazzi - Paradise\nJonas Brothers - Cool\nKhali...,8,90,1,[love their fans so much. they have really bee...,24,75,0


In [205]:
newtracks = newtracks.reset_index()
newtracks = newtracks.drop(columns=['index'])

In [172]:
newtracks

,Week,SongID,Song,Performer,Peak,WeeksOnChart,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,...,time_signature,debut,Lyrics,LyricPositive,LyricNeutral,LyricNegative,Tweets,TweetPositive,TweetNeutral,TweetNegative
0,05/15/20,DaisiesKaty Perry,Daisies,Katy Perry,NaN,None,False,173123,72,0.639,...,4.0,NaN,"Told them your dreams, and they all started la...",2,97,0,[oay im trying not to sleep but this is me rig...,57,42,0
1,05/15/20,Five More MinutesJonas Brothers,Five More Minutes,Jonas Brothers,NaN,None,False,150366,62,0.836,...,4.0,NaN,"Give me five more minutes\nBaby, I'm not finis...",2,97,0,[exactly 5 hours until x and ive more minutes!...,21,73,5
2,05/15/20,Lose SomebodyKygo,Lose Somebody,Kygo,NaN,None,False,199549,72,0.580,...,4.0,NaN,"It's a classic ""me"" mistake\nSomeone gives me ...",17,82,0,"[we’ve just added ose somebody by ygo, onerepu...",2,97,0
3,05/15/20,Play DateMelanie Martinez,Play Date,Melanie Martinez,NaN,None,True,179867,88,0.680,...,4.0,NaN,"You call me on the telephone, you feel so far ...",0,83,16,[ale usic to songs: 224 (+3) (new eak): lay da...,6,84,9
4,05/15/20,good guysLANY,good guys,LANY,NaN,None,False,224994,69,0.455,...,4.0,NaN,I wanna call you up\nTell you how I feel\nBut ...,8,91,0,"[i just wanna ive u everythin, show u i’m a so...",15,78,6
5,05/15/20,Stuck with U (with Justin Bieber)Ariana Grande,Stuck with U (with Justin Bieber),Ariana Grande,NaN,None,False,228482,92,0.597,...,3.0,NaN,"Mmm\nHey, yeah\n(That's just for fun)\n(What?)...",0,78,21,"[hey cn you plee ply ""tuck with u"" by nd i'...",12,82,5
6,05/15/20,Party GirlStaySolidRocky,Party Girl,StaySolidRocky,64.0,None,False,147800,86,0.728,...,4.0,84.0,"Girls just wanna have fun\nSo they party, so t...",46,53,0,"[don’t be cared to arty, don’t be cared to art...",7,89,3
7,05/15/20,Be Kind (with Halsey)Marshmello,Be Kind (with Halsey),Marshmello,NaN,None,False,172761,89,0.630,...,4.0,NaN,Drake - Pain 1993 (with Playboi Carti)\nMarshm...,10,84,5,[also e kind y halsey and arshello. literally ...,83,16,0
8,05/15/20,End this (L.O.V.E.)Hailee Steinfeld,End this (L.O.V.E.),Hailee Steinfeld,NaN,None,False,183218,70,0.621,...,4.0,NaN,L is for the way you lied to me\nO is I'm the ...,37,60,2,[l is for t way you lid to m o is i'm t only o...,6,91,2
9,05/15/20,X (with KAROL G)Jonas Brothers,X (with KAROL G),Jonas Brothers,NaN,None,False,185277,68,0.715,...,4.0,NaN,Bazzi - Paradise\nJonas Brothers - Cool\nKhali...,8,90,1,[love their fans so much. they have really bee...,24,75,0


In [173]:
topics = pd.read_csv("Data/LDA.csv")

In [174]:
def lemmatize_stemming(text):
    stemmer = PorterStemmer()
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

In [175]:
words = []
for lyrics in topics['Lyrics']:
    for word in gensim.utils.simple_preprocess(lyrics):
        if word not in gensim.parsing.preprocessing.STOPWORDS and len(word) > 3:
            words.append(lemmatize_stemming(word))
add_stop_words = [word for word, count in Counter(words).most_common() if count > 750]

In [176]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            token = lemmatize_stemming(token)
            if token not in add_stop_words:
                result.append(token)
    return result

In [177]:
processed_docs = topics['Lyrics'].map(preprocess)

In [179]:
dictionary = gensim.corpora.Dictionary(processed_docs)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [180]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [182]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [206]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
newtracks['Topic 0'] = 0.0
newtracks['Topic 1'] = 0.0
newtracks['Topic 2'] = 0.0
newtracks['Topic 3'] = 0.0
newtracks['Topic 4'] = 0.0
newtracks['Topic 5'] = 0.0
newtracks['Topic 6'] = 0.0
newtracks['Topic 7'] = 0.0
newtracks['Topic 8'] = 0.0
newtracks['Topic 9'] = 0.0

In [207]:
for idx, values in newtracks.iterrows():
    bow_vector = dictionary.doc2bow(preprocess(values['Lyrics']))
    for indx, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
        newtracks['Topic ' + str(indx)][idx] = score

/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [257]:
featureset = newtracks
featureset.dtypes

Week                          object
SongID                        object
Song                          object
Performer                     object
Peak                         float64
WeeksOnChart                 float64
spotify_track_explicit          bool
spotify_track_duration_ms      int64
spotify_track_popularity       int64
danceability                 float64
energy                       float64
key                           object
loudness                     float64
mode                         float64
speechiness                  float64
acousticness                 float64
instrumentalness             float64
liveness                     float64
valence                      float64
tempo                        float64
time_signature               float64
debut                        float64
Lyrics                        object
LyricPositive                  int64
LyricNeutral                   int64
LyricNegative                  int64
Tweets                        object
T

In [258]:
featureset = featureset.drop(columns=['Week', 'SongID', 'Song','Performer', 'Tweets', 'Lyrics'])

In [259]:
featureset['Classification'] = -1

In [260]:
for idx, values in featureset.iterrows():
    number = -1
    if values['Peak'] == 1:
        number = 0
    elif values['Peak'] < 6:
        number = 1
    elif values['Peak'] < 16:
        number = 2
    elif values['Peak'] < 31:
        number = 3
    elif values['Peak'] < 51:
        number = 4
    elif values['Peak'] < 76:
        number = 5
    elif values['Peak'] < 101:
        number = 6
    else:
        number = -1
    featureset['Classification'][idx] = number

/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [261]:
featureset = featureset.drop(columns=['Peak'])

In [262]:
for idx, values in featureset.iterrows():
    if isinstance(values['key'], int) == False:
        featureset['key'][idx] = -1

/Users/pk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [263]:
featureset['key']

0       1
1       4
2       1
3       5
4       0
5       8
6       6
7       4
8      11
9       4
10      3
11      0
12      6
13      4
14      5
15      6
16      1
17      9
18      1
19      6
20      9
21      8
22      5
23      2
24      1
25      6
26      1
27     10
28      8
29      7
       ..
106     1
107    10
108     0
109     4
110    -1
111     0
112     5
113     0
114     5
115     5
116     6
117     8
118     4
119     7
120     1
121    10
122     0
123     0
124     1
125     7
126     7
127     7
128     4
129     2
130     7
131     4
132     4
133    11
134     1
135     2
Name: key, Length: 136, dtype: object

In [214]:
featureset2 = pd.read_csv("Data/FeaturesLimited.csv")
featureset2 = featureset2.drop(columns=['Unnamed: 0'])

In [264]:
df=spark.createDataFrame(featureset2)
testdf = spark.createDataFrame(featureset)

In [265]:
RANDOM_SEED = 13579
transformed_df = df.rdd.map(lambda row: LabeledPoint(row[-1], Vectors.dense(row[0:-1])))
transformed_testdf = testdf.rdd.map(lambda row: LabeledPoint(row[-1], Vectors.dense(row[0:-1])))

# Training the Classifier
rf = RandomForest.trainClassifier(transformed_df, numClasses=7, categoricalFeaturesInfo={}, \
    numTrees=25, featureSubsetStrategy="auto", impurity="gini", \
    maxDepth=10, maxBins= 32, seed=RANDOM_SEED)

predictions = rf.predict(transformed_testdf.map(lambda x: x.features))
labels_and_predictions = transformed_testdf.map(lambda x: x.label).zip(predictions)
acc = labels_and_predictions.filter(lambda x: x[0] == x[1]).count() / float(transformed_testdf.count())
print("Model accuracy: %.3f%%" % (acc * 100))

Model accuracy: 0.735%


In [266]:
labels_and_predictions.take(135)

[(-1.0, 3.0),
 (-1.0, 1.0),
 (-1.0, 4.0),
 (-1.0, 2.0),
 (-1.0, 3.0),
 (-1.0, 0.0),
 (5.0, 6.0),
 (-1.0, 3.0),
 (-1.0, 4.0),
 (-1.0, 1.0),
 (-1.0, 2.0),
 (-1.0, 4.0),
 (-1.0, 2.0),
 (-1.0, 3.0),
 (-1.0, 3.0),
 (-1.0, 5.0),
 (-1.0, 2.0),
 (-1.0, 4.0),
 (-1.0, 2.0),
 (-1.0, 1.0),
 (-1.0, 2.0),
 (-1.0, 2.0),
 (-1.0, 4.0),
 (-1.0, 0.0),
 (-1.0, 3.0),
 (-1.0, 4.0),
 (-1.0, 2.0),
 (-1.0, 2.0),
 (-1.0, 3.0),
 (-1.0, 5.0),
 (-1.0, 2.0),
 (-1.0, 4.0),
 (-1.0, 4.0),
 (-1.0, 2.0),
 (-1.0, 4.0),
 (-1.0, 2.0),
 (-1.0, 2.0),
 (-1.0, 1.0),
 (-1.0, 1.0),
 (-1.0, 2.0),
 (-1.0, 2.0),
 (-1.0, 2.0),
 (-1.0, 2.0),
 (-1.0, 2.0),
 (-1.0, 3.0),
 (-1.0, 2.0),
 (-1.0, 2.0),
 (-1.0, 4.0),
 (-1.0, 3.0),
 (-1.0, 1.0),
 (-1.0, 2.0),
 (-1.0, 1.0),
 (-1.0, 4.0),
 (-1.0, 2.0),
 (-1.0, 5.0),
 (-1.0, 2.0),
 (-1.0, 2.0),
 (-1.0, 4.0),
 (-1.0, 2.0),
 (-1.0, 2.0),
 (-1.0, 2.0),
 (-1.0, 4.0),
 (-1.0, 3.0),
 (-1.0, 2.0),
 (-1.0, 1.0),
 (-1.0, 2.0),
 (-1.0, 1.0),
 (-1.0, 5.0),
 (-1.0, 4.0),
 (-1.0, 4.0),
 (-1.0, 3.0),
 (-1.0,

In [273]:
index = 0
hitsongs = []
predictionlist = predictions.collect()
for x in predictionlist:
    if x == 1.0:
        hitsongs.append(index)
    index+=1

In [274]:
hitsongs

[1, 9, 19, 37, 38, 49, 51, 64, 66, 73, 77, 82, 83, 90, 110, 112, 113, 114]

In [298]:
update = "One of these songs will rank in the Top 5 Songs of Next Week's Billboard Hot 100 Chart:\n\n"
sort = []
def takeSecond(elem):
    return elem[2]
for idx, values in newtracks.iterrows():
    if idx in hitsongs:
        sort.append([values['Song'], values['Performer'], values['spotify_track_popularity']])
sort = sorted(sort, key=takeSecond, reverse=True)
num = 0
for x in sort:
    if num < 5:
        update = update + x[0]+' by '+x[1]+'\n'
        num+=1
print(update)

One of these songs will rank in the Top 5 Songs of Next Week's Billboard Hot 100 Chart:

I’m Ready (with Demi Lovato) by Sam Smith
Secrets by Regard
Bloody Valentine by Machine Gun Kelly
head first by Christian French
Hold You Tonight (with Chris Lane) by Gryffin



In [277]:
auth = tweepy.OAuthHandler('TvrYJIPHns4OPJCetpYg2RJw4', 'juNWwVL0l527PMTMCni6NQaFB5e5Ksjf3DXBreSkGCjdTEjzYF')
auth.set_access_token('121839199-fJpZUrP3IS4R4MvoW8kaGg3PevusBjTSXu56mbe4', '5kAx7woytQWNRWuxuwgGAKX61onlluULLgPqHSXnfx6bW')

api = tweepy.API(auth)

In [299]:
api.update_status(update)

Status(_api=<tweepy.api.API object at 0x1a27605898>, _json={'created_at': 'Mon May 18 18:01:39 +0000 2020', 'id': 1262443247164686337, 'id_str': '1262443247164686337', 'text': "One of these songs will rank in the Top 5 Songs of Next Week's Billboard Hot 100 Chart:\n\nI’m Ready (with Demi Lovat… https://t.co/Hp2L70KLOl", 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/Hp2L70KLOl', 'expanded_url': 'https://twitter.com/i/web/status/1262443247164686337', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}]}, 'source': '<a href="https://github.com/pkhiyara/Predicting-The-Next-Hit-Song" rel="nofollow">Twitter Hit Song Predictor</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 121839199, 'id_str': '121839199', 'name': 'PK', 'screen_name': 'PunitKhiyara', 'location': '', 'descript

In [ ]:
time.sleep(604800)